In [12]:
#12/4/23 add for train set
#12/6/23 filter if ambiguous whether there is actually a water
import MDAnalysis
from MDAnalysis.analysis import distances
from MDAnalysis.analysis.hydrogenbonds.hbond_analysis import HydrogenBondAnalysis
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy
from scipy import stats

In [17]:
#############
#Run a bound water analysis for a set of structures
#Inputs:
#(a) pdb_path path to the pdb overall dir
#(b) list_pdbs list of pdb structures to work with
#(c) output_dir where to place output.npy
#(d) cutoff_d cutoff for bound water contact
#(e) name_job name of job
#(f) list_obsolete for train data 1 was obsolete
#############
def run_for_bd_water(pdb_path, list_pdbs, output_dir, cutoff_d, name_job, list_obsolete):
    
    #Keys- pdbs, values- resid lists and also average KD hydropathies
    w_info = {}
    
    w_remove_list = []
    
    #This is for removing from all
    w_remove_all_list = []
    
    #Iterate over each pdb
    for ipdb, pdb_analyze in enumerate(list_pdbs):
        
        print(f"On {pdb_analyze} index {ipdb}")
        
        #Obsolete filter
        if pdb_analyze not in list_obsolete:
        
            #############
            #Download PDBBind files, make universes
            #############
            protein_file = f"{pdb_path}/{pdb_analyze}/{pdb_analyze}_protein_processed.pdb"
            u_prot = MDAnalysis.Universe(protein_file)

            #Copied below from https://github.com/gcorso/DiffDock/blob/main/datasets/pdbbind.py
            #Only work with mol2 though revise because that is compatible with MDAnalysis- sdf is not
            for file in os.listdir(os.path.join(pdb_path, pdb_analyze)):
                if file.endswith(".mol2") and 'rdkit' not in file:
                    ligand_file = os.path.join(pdb_path, pdb_analyze, file)
                    #if lig is None and os.path.exists(os.path.join(pdb_path, pdb_analyze, file[:-4] + ".mol2")):  # read mol2 file if sdf file cannot be sanitized
                    #    print('Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.')
                    #    lig = read_molecule(os.path.join(pdb_path, pdb_analyze, file[:-4] + ".mol2"), remove_hs=False, sanitize=True)


            #ligand_file = obtaif"{pdb_path}/5zxk/5zxk_ligand.mol2"

            u_ligand = MDAnalysis.Universe(ligand_file)

            #Ref https://userguide.mdanalysis.org/stable/universe.html
            #Ref https://docs.mdanalysis.org/2.6.1/documentation_pages/core/universe.html#MDAnalysis.core.universe.Merge
            #Merge
            u_pl = MDAnalysis.core.universe.Merge(u_prot.atoms, u_ligand.atoms)

            #############
            #Protein and ligand info setup
            #############
            #For now- all segids allowed
            #May want to only have one later?
            psegids = list(set([a.segid for a in u_prot.atoms]))
            psegid_str = " ".join([p for p in psegids])
            #For now- all segids allowed
            #May want to only have one later?
            #psegids = list(set([a.segid for a in u_prot.atoms]))
            #if len(psegids) > 1:
            #    print("over 1 protein segids")
            #    print(psegids)
            #psegid = psegids[0]
            lresnames = list(set([a.resname for a in u_ligand.atoms]))

            #for now let only 1 ligand resname be used, simpifies
            if len(lresnames) > 1:
                print("over 1 ligand resnames REMOVAL")
                print(lresnames)
                print(f"resname string {lresname}")
                w_remove_list.append(pdb_analyze)

            else:
                lresname = lresnames[0]

                #If ligand resname is a protein resid- exclude
                if lresname in ["ALA",
                "CYS",
                "ASP",
                "GLU",
                "PHE",
                "GLY",
                "HIS",
                "ILE",
                "LYS",
                "LEU",
                "MET",
                "ASN",
                "PRO",
                "GLN",
                "ARG",
                "SER",
                "THR",
                "VAL",
                "TRP",
                "TYR",
                "MSE"]:
                    w_remove_list.append(pdb_analyze)
                    print("Ligand resname is protein resname: REMOVAL")

                else:

                    #############
                    #Find nearby waters in pdb
                    #############
                    #Now pull from rcsb
                    #Ref https://docs.mdanalysis.org/stable/documentation_pages/coordinates/MMTF.html
                    #rcsb_entry = "5ZXK"
                    u_rcsb = MDAnalysis.coordinates.MMTF.fetch_mmtf(pdb_analyze)

                    u_rcsb_protein_sel = f"(protein and segid {psegid_str})"
                    u_rcsb_ligand_sel = f"(resname {lresname})"
                    p_near_l_sel = f"{u_rcsb_protein_sel} and around 3.3 {u_rcsb_ligand_sel}"
                    l_near_p_sel = f"{u_rcsb_ligand_sel} and around 3.3 {u_rcsb_protein_sel}"
                    #print("p_near_l_sel")
                    #print(p_near_l_sel)
                    #print("l_near_p_sel")
                    #print(l_near_p_sel)

                    #TODO confirm always water is named HOH- now simply ignoring if no HOH in system
                    water_atoms = u_rcsb.select_atoms("resname HOH")
                    if len(water_atoms) < 1:
                        print(f"NO WATERS PDB {pdb_analyze} REMOVAL")
                        w_remove_list.append(pdb_analyze)

                    #If waters are in system- find # nearby and record
                    else:
                        #print(f"resname HOH and around {cutoff_d} ((protein and segid {psegid_str}) and (resname {lresname}))")
                        nearby_water_sel = u_rcsb.select_atoms(f"resname HOH and (around {cutoff_d} (protein and segid {psegid_str})) and (around {cutoff_d} (resname {lresname}))")
                        #print(nearby_water_sel)

                        unique_w_resids = list(set([r.resid for r in nearby_water_sel]))
                        #print("unique resids")
                        #print(unique_w_resids)

                        #record
                        w_info[pdb_analyze] = len(unique_w_resids)

                #############
                #H bond calc- not yet doing
                #############
                '''
                #Ref https://userguide.mdanalysis.org/stable/universe.html
                #Ref https://docs.mdanalysis.org/2.6.1/documentation_pages/core/universe.html#MDAnalysis.core.universe.Merge
                #Merge
                u_pl = MDAnalysis.core.universe.Merge(u_prot.atoms, u_ligand.atoms)

                #Run HB analysis
                #Ref https://docs.mdanalysis.org/stable/documentation_pages/analysis/hydrogenbonds.html
                #HB create
                plhbonds = MDAnalysis.analysis.hydrogenbonds.hbond_analysis.HydrogenBondAnalysis(
                  universe=u_pl,
                  between=["protein", f"resname {lresname}"]
                  )

                #Protein and ligand selections
                protein_hydrogens_sel = plhbonds.guess_hydrogens("protein")
                protein_acceptors_sel = hbonds.guess_acceptors(u_rcsb_protein_sel)

                l_hydrogens_sel = hbonds.guess_hydrogens(f"resname {lresname}")
                l_acceptors_sel = hbonds.guess_acceptors(f"resname {lresname}")

                #Combine
                plhbonds.hydrogens_sel = f"({u_rcsb_protein_sel} and ({protein_hydrogens_sel})) or ({u_rcsb_ligand_sel} and ({l_hydrogens_sel}))"
                plhbonds.acceptors_sel = f"({u_rcsb_protein_sel} and ({protein_acceptors_sel})) or ({u_rcsb_ligand_sel} and ({l_acceptors_sel}))"

                print("H sel")
                print(plhbonds.hydrogens_sel)

                print("Acc sel")
                print(plhbonds.hydrogens_sel)

                plhbonds.run()
                print(plhbonds)
                '''
        
    #Save this info
    np.save(f"{output_dir}/Nearby_W_Cutoff_{name_job}_{cutoff_d}.npy", w_info)
    
    np.save(f"{output_dir}/REMOVAL_Nearby_W_Cutoff_{name_job}_{cutoff_d}.npy", w_remove_list)
    np.save(f"{output_dir}/ALLREMOVAL_Nearby_W_Cutoff_{name_job}_{cutoff_d}.npy", w_remove_all_list)
    
    print("REMOVALS")
    print(w_remove_list)
    print(f"REMOVALS ARE {len(w_remove_list)} out of {len(list_pdbs)}")
    print("ALL REMOVALS")
    print(w_remove_list)
    print(f"ALL REMOVALS ARE {len(w_remove_list)} out of {len(list_pdbs)}")

In [18]:
pdb_dir = "/Users/dsharon/Documents/MIT/6.8701/Project/Code/HarmonicFlow/FlowSite/data/PDBBind_processed"
pdb_list = "/Users/dsharon/Documents/MIT/6.8701/Project/Data/From_Hannes/TEST3_top40_epoch75_FILTER_restart_cacheNewRestart_big_ema_ESM2emb_tr34_WITH_fixedSamples28_id1_FILTERFROM_temp_restart_ema_ESM2emb_tr34/complex_names.npy"
odir = "/Users/dsharon/Documents/MIT/6.8701/Project/Analysis/RMSD_and_Chem_Feats/Site_Comp_231206"
dcutoff = 3.70
jname = "DiffDock"

#From Hannes
with open(pdb_list, 'rb') as f:
    complex_names = np.load(f)

run_for_bd_water(pdb_dir, complex_names, odir, dcutoff, jname, [])

On 6qqw index 0
On 6d08 index 1
Ligand resname is protein resname: REMOVAL
On 6jap index 2
On 6np2 index 3
On 6uvp index 4
On 6oxq index 5
On 6jsn index 6
On 6hzb index 7
Ligand resname is protein resname: REMOVAL
On 6qrc index 8
On 6oio index 9
On 6jag index 10
On 6moa index 11
On 6hld index 12
On 6i9a index 13
On 6e4c index 14
On 6g24 index 15
On 6jb4 index 16
On 6s55 index 17
On 6seo index 18
On 6dyz index 19
On 5zk5 index 20
Ligand resname is protein resname: REMOVAL
On 6jid index 21
On 5ze6 index 22
On 6qlu index 23
On 6a6k index 24
On 6qgf index 25
On 6e3z index 26
On 6te6 index 27
On 6pka index 28
On 6g2o index 29
On 6jsf index 30
On 5zxk index 31
On 6qxd index 32
On 6n97 index 33
On 6jt3 index 34
On 6qtr index 35
Ligand resname is protein resname: REMOVAL
On 6oy1 index 36
On 6n96 index 37
On 6qzh index 38
On 6qqz index 39
On 6qmt index 40
On 6ibx index 41
On 6hmt index 42
Ligand resname is protein resname: REMOVAL
On 5zk7 index 43
On 6k3l index 44
On 6cjs index 45
On 6n9l index

On 6mo2 index 362
NO WATERS PDB 6mo2 REMOVAL
REMOVALS
['6d08', '6hzb', '5zk5', '6qtr', '6hmt', '6ott', '6hlb', '6n0m', '6v5l', '6kqi', '6i41', '6iql', '6hza', '6qtq', '6os5', '6i68', '6mo0', '6d07', '6qts', '6i5p', '6qsz', '6hhp', '6hzd', '6qtx', '6o3y', '6rr0', '6o0h', '6o3x', '6o9c', '6d3z', '6hle', '6n4b', '6qtw', '6hv2', '6hzc', '6qtm', '6oie', '6d3y', '6o9b', '6d40', '6i7a', '6os6', '6dql', '6q4q', '6d3x', '6mo2']
REMOVALS ARE 46 out of 363
ALL REMOVALS
['6d08', '6hzb', '5zk5', '6qtr', '6hmt', '6ott', '6hlb', '6n0m', '6v5l', '6kqi', '6i41', '6iql', '6hza', '6qtq', '6os5', '6i68', '6mo0', '6d07', '6qts', '6i5p', '6qsz', '6hhp', '6hzd', '6qtx', '6o3y', '6rr0', '6o0h', '6o3x', '6o9c', '6d3z', '6hle', '6n4b', '6qtw', '6hv2', '6hzc', '6qtm', '6oie', '6d3y', '6o9b', '6d40', '6i7a', '6os6', '6dql', '6q4q', '6d3x', '6mo2']
ALL REMOVALS ARE 46 out of 363


In [19]:
#12/1/23 HF run and add argument for name
pdb_dir = "/Users/dsharon/Documents/MIT/6.8701/Project/Code/HarmonicFlow/FlowSite/data/PDBBind_processed"
pdb_list = "/Users/dsharon/Documents/MIT/6.8701/Project/Data/From_Hannes/HarmonicFlow_complex_names.npy"
odir = "/Users/dsharon/Documents/MIT/6.8701/Project/Analysis/RMSD_and_Chem_Feats/Site_Comp_231206"
dcutoff = 3.70
jname = "HF"

#From Hannes
with open(pdb_list, 'rb') as f:
    complex_names = np.load(f)

run_for_bd_water(pdb_dir, complex_names, odir, dcutoff, jname, [])

On 6jt3 index 0
On 6jbb index 1
On 6ufo index 2
On 6os6 index 3
Ligand resname is protein resname: REMOVAL
On 6jbe index 4
On 6qra index 5
On 6qsz index 6
Ligand resname is protein resname: REMOVAL
On 6jam index 7
On 6a87 index 8
On 6oie index 9
Ligand resname is protein resname: REMOVAL
On 6n4b index 10
NO WATERS PDB 6n4b REMOVAL
On 6dyz index 11
On 6hhh index 12
On 6i5p index 13
Ligand resname is protein resname: REMOVAL
On 6e6v index 14
On 6o9c index 15
Ligand resname is protein resname: REMOVAL
On 6c85 index 16
On 6e13 index 17
On 6cjj index 18
On 6nri index 19
On 6mja index 20
On 6qr7 index 21
On 6o0h index 22
NO WATERS PDB 6o0h REMOVAL
On 6qmt index 23
On 6ibz index 24
On 6d3y index 25
Ligand resname is protein resname: REMOVAL
On 6qr0 index 26
On 6s9w index 27
On 6p8x index 28
On 6s07 index 29
On 6fe5 index 30
On 6oxp index 31
On 5zlf index 32
On 6n8x index 33
On 6qtw index 34
Ligand resname is protein resname: REMOVAL
On 6mhd index 35
On 6jut index 36
On 6uhu index 37
On 6i8m i

In [20]:
#12/4/23 train set run
pdb_dir = "/Users/dsharon/Documents/MIT/6.8701/Project/Code/HarmonicFlow/FlowSite/data/PDBBind_processed"
pdb_list = "/Users/dsharon/Documents/MIT/6.8701/Project/Data/From_Hannes/DiffDock_PDB_Train/timesplit_no_lig_overlap_train.npy"
odir = "/Users/dsharon/Documents/MIT/6.8701/Project/Analysis/RMSD_and_Chem_Feats/Site_Comp_231206"
dcutoff = 3.70
jname = "TrainSet"
obsolete_list = ["5ab1", "4jdf",  "1qon", "4pox"] #Obsolete

#From Hannes
with open(pdb_list, 'rb') as f:
    complex_names = np.load(f)

run_for_bd_water(pdb_dir, complex_names, odir, dcutoff, jname, obsolete_list)

On 3dpf index 0
On 2zy1 index 1
On 6h77 index 2
On 5c0m index 3
Ligand resname is protein resname: REMOVAL
On 4o2c index 4
On 3a2c index 5
NO WATERS PDB 3a2c REMOVAL
On 3b2t index 6
On 1v2q index 7
On 4q87 index 8
On 4j28 index 9
On 3efw index 10
On 4mi6 index 11
On 3k8c index 12
On 3s54 index 13
On 2fda index 14
On 2q55 index 15
On 5ylv index 16
On 4ozo index 17
On 3fzn index 18
On 3eoc index 19
NO WATERS PDB 3eoc REMOVAL
On 4tn2 index 20
On 3hfv index 21
On 5gg4 index 22
Ligand resname is protein resname: REMOVAL
On 5svi index 23
Ligand resname is protein resname: REMOVAL
On 2r58 index 24
On 5d1n index 25
On 4yqv index 26
On 1hs6 index 27
On 3zxh index 28
On 1ft7 index 29
On 4hww index 30
On 6b1j index 31
On 3di6 index 32
On 5jaz index 33
On 5ema index 34
Ligand resname is protein resname: REMOVAL
On 6gip index 35
On 5am5 index 36
On 4bnt index 37
On 1fq6 index 38
On 3pqz index 39
Ligand resname is protein resname: REMOVAL
On 6eil index 40
On 2q11 index 41
On 4nw7 index 42
On 4ahr in

On 4pks index 337
On 3nf3 index 338
Ligand resname is protein resname: REMOVAL
On 2oaz index 339
On 3pwk index 340
On 3ryy index 341
On 2y7k index 342
On 6dm8 index 343
On 5ism index 344
On 5i7u index 345
On 2qtn index 346
On 5cuu index 347
On 5n1y index 348
Ligand resname is protein resname: REMOVAL
On 4kot index 349
On 5wa1 index 350
Ligand resname is protein resname: REMOVAL
On 2wyi index 351
On 188l index 352
On 3h2o index 353
On 1sw1 index 354
On 1zuc index 355
On 2nnq index 356
On 4l8m index 357
On 5j75 index 358
On 4afe index 359
On 5nwi index 360
Ligand resname is protein resname: REMOVAL
On 2n06 index 361
NO WATERS PDB 2n06 REMOVAL
On 3l4w index 362
On 3ibl index 363
On 4l31 index 364
On 4ruy index 365
On 4ufm index 366
On 5sxk index 367
NO WATERS PDB 5sxk REMOVAL
On 4hnn index 368
Ligand resname is protein resname: REMOVAL
On 5q0x index 369
On 4zij index 370
On 3fdn index 371
On 5fe6 index 372
On 3nuj index 373
On 5h8x index 374
On 5ets index 375
On 4x5z index 376
On 1w3k ind

/Users/dsharon/anaconda3/envs/mda_env_2/lib/python3.11/site-packages/MDAnalysis/topology/MOL2Parser.py:234: UserWarning: Unknown elements found for some atoms: {'Pt'}. These have been given an empty element record.
  warnings.warn("Unknown elements found for some "


On 3zi8 index 380
On 1nl4 index 381
NO WATERS PDB 1nl4 REMOVAL
On 5cao index 382
On 4gjd index 383
On 2p3d index 384
On 2z8e index 385
On 3w2p index 386
On 1f28 index 387
On 2iw6 index 388
On 1h9l index 389
Ligand resname is protein resname: REMOVAL
On 4o78 index 390
On 1soj index 391
On 3krw index 392
On 5g4n index 393
On 3t4v index 394
On 5drr index 395
On 5o0a index 396
On 4c4i index 397
On 2vwx index 398
On 6miq index 399
On 6guk index 400
On 5c3k index 401
On 5fhn index 402
On 3o75 index 403
On 5kap index 404
On 4zo5 index 405
On 2jko index 406
On 3iss index 407
On 4eg6 index 408
On 4fcd index 409
On 3v7s index 410
On 4bcf index 411
On 2qyn index 412
On 1ikx index 413
On 6f6i index 414
On 5os5 index 415
On 1om2 index 416
Ligand resname is protein resname: REMOVAL
On 6axb index 417
On 3s3n index 418
On 1fl6 index 419
NO WATERS PDB 1fl6 REMOVAL
On 4luo index 420
On 1qiw index 421
NO WATERS PDB 1qiw REMOVAL
On 5f2w index 422
On 3udj index 423
On 5tbj index 424
On 5k0c index 425
On 5n

/Users/dsharon/anaconda3/envs/mda_env_2/lib/python3.11/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: CD
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


...

On 2xi7 index 14824
On 1wqv index 14825
On 2b8v index 14826
On 6f92 index 14827
On 4jve index 14828
On 3oy0 index 14829
On 6h41 index 14830
Ligand resname is protein resname: REMOVAL
On 4zx4 index 14831
On 4q15 index 14832
On 6czv index 14833
On 2wti index 14834
On 5j9l index 14835
On 1o5r index 14836
On 5n1v index 14837
On 2bq7 index 14838
On 4fic index 14839
On 2ole index 14840
On 3umw index 14841
On 1a1e index 14842
On 1ax0 index 14843
On 4ux9 index 14844
Ligand resname is protein resname: REMOVAL
On 6bhi index 14845
On 3pj3 index 14846
On 5m55 index 14847
On 2uw6 index 14848
On 2bqv index 14849
On 2ea2 index 14850
On 5dl1 index 14851
NO WATERS PDB 5dl1 REMOVAL
On 6g7f index 14852
On 5o8t index 14853
On 5nze index 14854
On 5fkj index 14855
On 5t1l index 14856
Ligand resname is protein resname: REMOVAL
On 4wj7 index 14857
Ligand resname is protein resname: REMOVAL
On 3ibi index 14858
On 5h22 index 14859
On 4bgh index 14860
On 2j9h index 14861
On 2iyf index 14862
On 5m28 index 14863
O

On 1q9d index 15143
On 4idt index 15144
On 4k2f index 15145
On 2whp index 15146
On 3snc index 15147
On 4otg index 15148
On 2lty index 15149
Ligand resname is protein resname: REMOVAL
On 5bnr index 15150
On 6c7r index 15151
On 3as3 index 15152
On 4ge9 index 15153
On 4xg7 index 15154
On 5dw1 index 15155
On 3ivq index 15156
Ligand resname is protein resname: REMOVAL
On 2qrg index 15157
On 2xdk index 15158
On 1gno index 15159
On 2gg5 index 15160
On 1d4y index 15161
On 5dqe index 15162
On 3rwi index 15163
Ligand resname is protein resname: REMOVAL
On 4x8o index 15164
On 3el9 index 15165
On 4kup index 15166
On 3eks index 15167
On 2kff index 15168
Ligand resname is protein resname: REMOVAL
On 1w5y index 15169
On 2flh index 15170
On 2vwv index 15171
On 3rvg index 15172
On 4oeu index 15173
Ligand resname is protein resname: REMOVAL
On 3u4o index 15174
On 2pv3 index 15175
Ligand resname is protein resname: REMOVAL
On 4i0t index 15176
On 3aru index 15177
On 4yv0 index 15178
On 5ura index 15179
On

On 4ahu index 15455
On 1gx4 index 15456
On 3wi2 index 15457
On 4de2 index 15458
On 5g2g index 15459
On 2wuu index 15460
On 5nud index 15461
On 4hcu index 15462
On 3k37 index 15463
On 6prc index 15464
On 2e7f index 15465
On 4bb9 index 15466
On 1amk index 15467
On 4l1a index 15468
On 4lqy index 15469
On 4a16 index 15470
On 5wkf index 15471
Ligand resname is protein resname: REMOVAL
On 4qsm index 15472
On 3sxu index 15473
Ligand resname is protein resname: REMOVAL
On 3uqg index 15474
On 5lif index 15475
On 4azf index 15476
On 5ntt index 15477
On 3vsw index 15478
NO WATERS PDB 3vsw REMOVAL
On 5fos index 15479
Ligand resname is protein resname: REMOVAL
On 1qin index 15480
On 3e7b index 15481
On 1ll4 index 15482
On 3vjm index 15483
On 3ny3 index 15484
Ligand resname is protein resname: REMOVAL
On 6aqs index 15485
On 1k6c index 15486
On 4ngr index 15487
On 6fc6 index 15488
Ligand resname is protein resname: REMOVAL
On 2gnj index 15489
On 4zy6 index 15490
On 3u8d index 15491
On 2aou index 1549

On 3eq8 index 15781
On 3q44 index 15782
On 4ago index 15783
On 5e2p index 15784
On 1pxj index 15785
On 4p6c index 15786
On 1zoh index 15787
On 4i7j index 15788
On 4hlg index 15789
On 5j6n index 15790
On 3s2o index 15791
On 4ec0 index 15792
On 3wiy index 15793
On 4k77 index 15794
On 5yf1 index 15795
On 1hsl index 15796
Ligand resname is protein resname: REMOVAL
On 4tjz index 15797
On 6cbh index 15798
On 6g38 index 15799
On 4efu index 15800
On 5wep index 15801
NO WATERS PDB 5wep REMOVAL
On 1jwm index 15802
Ligand resname is protein resname: REMOVAL
On 3wfg index 15803
On 5lvl index 15804
On 5kqd index 15805
NO WATERS PDB 5kqd REMOVAL
On 1fq5 index 15806
On 5nn6 index 15807
On 3b5r index 15808
On 5xag index 15809
On 3w69 index 15810
On 3udr index 15811
On 4c4h index 15812
On 5mgi index 15813
On 5ad1 index 15814
On 5zh3 index 15815
NO WATERS PDB 5zh3 REMOVAL
On 4i7b index 15816
Ligand resname is protein resname: REMOVAL
On 6cwi index 15817
On 1tkb index 15818
NO WATERS PDB 1tkb REMOVAL
On 

On 5xvf index 16094
On 1xka index 16095
On 4ewn index 16096
On 4ewr index 16097
Ligand resname is protein resname: REMOVAL
On 4a7b index 16098
On 2xab index 16099
On 6g9b index 16100
On 4u69 index 16101
On 1qb6 index 16102
On 3c6t index 16103
On 3qar index 16104
On 4k5o index 16105
On 2gnf index 16106
On 1u3q index 16107
On 3g31 index 16108
On 3oev index 16109
NO WATERS PDB 3oev REMOVAL
On 4oc2 index 16110
On 1eby index 16111
On 3uef index 16112
Ligand resname is protein resname: REMOVAL
On 4k0o index 16113
On 5ehi index 16114
On 5h63 index 16115
On 3wcg index 16116
On 6hvi index 16117
On 1hii index 16118
On 4jhq index 16119
On 3sgt index 16120
On 4aua index 16121
On 4hnc index 16122
On 3bxh index 16123
On 5g5v index 16124
On 4j08 index 16125
On 4anu index 16126
On 3hac index 16127
On 2vcq index 16128
On 2f7p index 16129
On 4ycl index 16130
On 3s22 index 16131
On 3obq index 16132
Ligand resname is protein resname: REMOVAL
On 1o9k index 16133
Ligand resname is protein resname: REMOVAL
O

In [10]:
r1 = ['6d08', '6hzb', '5zk5', '6qtr', '6hmt', '6ott', '6hlb', '6n0m', '6v5l', '6kqi', '6i41', '6iql', '6hza', '6qtq', '6os5', '6i68', '6mo0', '6d07', '6qts', '6i5p', '6qsz', '6hhp', '6hzd', '6qtx', '6o3y', '6rr0', '6o0h', '6o3x', '6o9c', '6d3z', '6hle', '6n4b', '6qtw', '6hv2', '6hzc', '6qtm', '6oie', '6d3y', '6o9b', '6d40', '6i7a', '6os6', '6dql', '6q4q', '6d3x', '6mo2']
r2 = ['6os6', '6qsz', '6oie', '6n4b', '6i5p', '6o9c', '6o0h', '6d3y', '6qtw', '6qts', '6mo2', '6mo0', '6qtr', '6rr0', '6hzd', '6os5', '6d07', '6i41', '6n0m', '6d40', '6i7a', '6o3y', '6hhp', '6qtq', '6ott', '6dql', '6o9b', '6hle', '6q4q', '6qtx', '6d3x', '6d3z', '6hza', '6d08', '6qtm', '6hmt', '6hv2', '6v5l', '6i68', '6hlb', '6o3x', '5zk5', '6hzc', '6hzb']
print([r1c for r1c in r1 if r1c not in r2])
print([r2c for r2c in r2 if r2c not in r1])

['6kqi', '6iql']
[]


In [11]:
sorted(r1)

['5zk5',
 '6d07',
 '6d08',
 '6d3x',
 '6d3y',
 '6d3z',
 '6d40',
 '6dql',
 '6hhp',
 '6hlb',
 '6hle',
 '6hmt',
 '6hv2',
 '6hza',
 '6hzb',
 '6hzc',
 '6hzd',
 '6i41',
 '6i5p',
 '6i68',
 '6i7a',
 '6iql',
 '6kqi',
 '6mo0',
 '6mo2',
 '6n0m',
 '6n4b',
 '6o0h',
 '6o3x',
 '6o3y',
 '6o9b',
 '6o9c',
 '6oie',
 '6os5',
 '6os6',
 '6ott',
 '6q4q',
 '6qsz',
 '6qtm',
 '6qtq',
 '6qtr',
 '6qts',
 '6qtw',
 '6qtx',
 '6rr0',
 '6v5l']